In [1]:
!pip install --quiet torch==2.6.0 darts==0.33.0 scikit-learn==1.6.1 2>/dev/null

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 972.2/972.2 kB 17.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 91.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 85.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 7.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Imports

In [2]:
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.signal import savgol_filter

import torch
from darts import TimeSeries
from darts.models import NHiTSModel
from darts.utils.model_selection import train_test_split

# Suppress warnings and set figure size
warnings.filterwarnings("ignore")
plt.rcParams['figure.figsize'] = (12, 5)
plt.style.use('fivethirtyeight')

In [3]:
class Optimization:
    """
    End-to-end trigger-reconstruction helper for the “Trojan Horse Hunt” baseline.

    The class wraps a pre-trained **N-HiTS** model and a clean validation snippet,
    then offers utilities to:

    1. **Probe** the model with synthetic spikes and infer which telemetry channels
       are trojaned.
    2. **Optimise** an additive time series trigger (one per poisoned channel) that
       maximises forecast deviation while remaining time-local.
    3. **Inspect** optimisation logs, smooth / post-process the trigger, convert it
       into the competition’s `(3, 75)` format, and plot qualitative diagnostics.

    Parameters
    ----------
    model : NHiTSModel
        A fitted darts.models.NHiTSModel that may contain a trojan.
    val_clean : TimeSeries
        Clean validation slice used for probing and as the optimisation baseline.
    insert_pos : int, default=200
        Index at which the trigger is inserted into the input series.
    trigger_duration : int, default=75
        Length (in timesteps) of the additive trigger.
    lambda_reg : float, default=0.5
        Weight on the ℓ² regularisation term of the trigger (`||δ||₂`).
    alpha_reg : float, default=1.5
        Weight on the *tracking* loss (forces forecast to follow the trigger).
    beta_reg : float, default=2
        Weight on the *difference* loss (distance between poisoned and clean
        forecasts).
    epochs : int, default=100
        Number of optimisation steps per channel.
    forecast_horizon : int, default=400
        Length of the model forecast used in the loss computations.
    input_chunk_length : int, default=400
        Number of historical points fed to the model.
    """
    def __init__(self, model: NHiTSModel,
                 val_clean: TimeSeries,
                 insert_pos: int = 200,
                 trigger_duration = 75,
                 lambda_reg: float = 0.5,
                 alpha_reg: float = 1.5,
                 beta_reg: float = 2,
                 epochs: int = 300,
                 forecast_horizon: int = 400,
                 input_chunk_length: int = 400):
        
        self.model = model
        self.val_clean = val_clean
        self.insert_pos = insert_pos
        self.trigger_duration = trigger_duration
        self.lambda_reg = lambda_reg
        self.alpha_reg = alpha_reg
        self.beta_reg = beta_reg
        self.epochs = epochs
        self.forecast_horizon = forecast_horizon
        self.input_chunk_length = input_chunk_length
        self.channels = ["channel_44", "channel_45", "channel_46"]


    def get_poisoned_channels(self, spike_value: float):
        """
        Heuristically identify which of the three telemetry channels are trojaned.

        A short spike (`spike_value`) is injected into each candidate channel.
        If the resulting forecast exceeds a channel-specific min/max tolerance
        band, that channel is flagged as *poisoned*.

        Parameters
        ----------
        spike_value : float
            Amplitude of the synthetic spike inserted between indices 250–259.

        Returns
        -------
        list[str]
            Channel names (e.g. ``["channel_44", "channel_46"]``) whose forecasts
            react anomalously and are therefore suspected to carry a back-door.
        """

        val_probed_df_copy = self.val_clean[:self.input_chunk_length].pd_dataframe().copy()
        for channel in self.channels:
            val_probed_df_copy[channel].iloc[250:260] = spike_value
        val_probed_df_copy = TimeSeries.from_dataframe(val_probed_df_copy.astype("float32"))

        val_clean_df_copy = self.val_clean[:self.input_chunk_length].pd_dataframe().copy()
        stat = val_clean_df_copy.describe()
        
        poisoned_channels = []

        for channel in self.channels:
            more_than_max = (stat[channel]["max"] + 0.3*(stat[channel]["max"] - stat[channel]["min"])) > self.model.predict(400, series=val_probed_df_copy, num_samples=1)[channel].values()
            any_more_than_max = not more_than_max.all()
            
            less_than_min = (stat[channel]["min"] - 0.3*(stat[channel]["max"] - stat[channel]["min"])) < self.model.predict(400, series=val_probed_df_copy, num_samples=1)[channel].values()
            any_less_than_min = not less_than_min.all()

            if (any_more_than_max or any_less_than_min):
                poisoned_channels.append(channel)
        return poisoned_channels



    def get_num_poisoned_channels(self, poisoned_channels: list[str]):
        """
        Map poisoned channel names to their positional indices (0, 1, 2).

        Parameters
        ----------
        poisoned_channels : list[str]
            Output of :py:meth:`get_poisoned_channels`.

        Returns
        -------
        list[int]
            Zero-based indices corresponding to ``val_clean.components``.
        """
        """
        Returns a list of positions (0-based indices) of poisoned channels in val_poisoned.
        """
        val_poisoned_channels = list(self.val_clean.components)
        return [val_poisoned_channels.index(ch) for ch in poisoned_channels if ch in val_poisoned_channels]

    
    def create_input_tensor(self):
        """
        Convert the first ``input_chunk_length`` timesteps of ``val_clean`` into
        a Torch tensor of shape ``(T, 3)``.

        Returns
        -------
        torch.Tensor
            Clean input ready for in-place trigger injection.
        """
        clean_input_np = self.val_clean[:self.input_chunk_length].values()
        input_tensor = torch.tensor(clean_input_np, dtype=torch.float32)
        return input_tensor
    
    def create_clean_input(self):
        """
        Convenience wrapper that clones the clean input tensor and converts it
        back to a :class:`darts.TimeSeries`. Mainly used when a detached copy of
        the untouched sequence is required.
        """
        clean = self.create_input_tensor().clone()
        clean = TimeSeries.from_values(clean.detach().numpy())

    def discover_trigger_injection(self, 
                                   input_tensor: torch.Tensor, 
                                   channel: int, 
                                   epochs: int=250):
        """
        Gradient-search for an additive trigger on a single channel.

        A learnable vector ``δ ∈ ℝ^{trigger_duration}`` is inserted at
        ``insert_pos`` on the specified channel and optimised with an AdamW
        + StepLR schedule.

        Loss
        ----
        ``loss = -β·diff_loss  - λ·‖δ‖₂  + α·tracking_loss``

        * **diff_loss** – L1 distance between poisoned and clean forecasts  
        * **tracking_loss** – L1 distance between poisoned forecast and the
          trigger values themselves (encourages the forecast to “follow” δ)  
        * **‖δ‖₂** – ℓ² size penalty

        Parameters
        ----------
        input_tensor : torch.Tensor
            Clean input series on which to graft the trigger.
        channel : int
            Index (0, 1, 2) of the candidate poisoned channel.
        epochs : int, default=200
            Optimisation iterations.

        Returns
        -------
        tuple[np.ndarray, TimeSeries, pandas.DataFrame]
            1. Optimised trigger δ (shape ``(trigger_duration,)``)  
            2. Modified input series with δ applied (TimeSeries)  
            3. Per-epoch log of ``tracking_loss``, ``diff_loss``, ``reg_loss``
        """
  
        delta = torch.zeros((self.trigger_duration, 1), dtype=torch.float32, requires_grad=True)
        optimizer = torch.optim.Adam([delta], lr=0.2, weight_decay=1e-4)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.9)

        for i in tqdm(range(epochs)):

            optimizer.zero_grad()
            modified = input_tensor.clone()
            modified[self.insert_pos:self.insert_pos + self.trigger_duration, channel] += delta.squeeze()
            modified_series = TimeSeries.from_values(modified.detach().numpy())

            
            pred_poisoned = self.model.predict(n=self.forecast_horizon, series=modified_series, verbose=False)
            pred_clean = self.model.predict(n=self.forecast_horizon, series=self.val_clean[:self.input_chunk_length], verbose=False)
            poisoned_tensor = torch.tensor(pred_poisoned.values(), dtype=torch.float32)
            clean_tensor = torch.tensor(pred_clean.values(), dtype=torch.float32)

            # for guess_channel in guess_poisoned_channel:
            tracking_target = modified[-self.forecast_horizon:, channel] 

            tracking_loss = torch.sum(torch.abs(poisoned_tensor[:, channel] - tracking_target))
            diff_loss = torch.sum(torch.abs(poisoned_tensor[:, channel] - clean_tensor[:, channel]))
            reg_loss =  torch.norm(delta, p=2)
            
            loss = - self.beta_reg * diff_loss \
                - self.lambda_reg * reg_loss \
                + self.alpha_reg * tracking_loss
                
            discovered_trigger = delta.detach().numpy().flatten()
        
            # Log metrics to a DataFrame after each iteration
            if 'opt_log_df' not in locals():
                opt_log_df = pd.DataFrame(columns=["epoch", "tracking_loss", "diff_loss", "reg_loss"])
            opt_log_df.loc[len(opt_log_df)] = [i, tracking_loss.item(), diff_loss.item(), reg_loss.item()]

            if i != epochs - 1:
                loss.backward()
                optimizer.step()
                scheduler.step()

        modified = input_tensor.clone()
        modified[self.insert_pos:self.insert_pos + self.trigger_duration, channel] += delta.squeeze()
        modified_series = TimeSeries.from_values(modified.detach().numpy())
        
        return delta.detach().numpy().flatten(), modified_series, opt_log_df
    
    def find_best_trigger(self, opt_log_df: pd.DataFrame,
                          poisoned_channels: list[str],
                          target_preds_diff: float = 2.5,
                          target_context_pred_diff: float = 4,
                          target_reg: float = 0.075):
        """
        Rank optimisation epochs by distance to hand-tuned target loss values.

        A Euclidean distance is computed in the 3-D space
        ``(diff_loss, tracking_loss, reg_loss)`` with a ×10 weight on reg_loss
        to keep the trigger small.

        Returns
        -------
        pandas.DataFrame
            ``opt_log_df`` sorted ascending by the new column ``loss_distance``.
        """

        
        ## global targets used for finding the best trigger
        opt_log_df['loss_distance'] = np.sqrt((opt_log_df['diff_loss'] - target_preds_diff)**2 \
                                    + (opt_log_df['tracking_loss'] - target_context_pred_diff)**2 \
                                    + 10*(opt_log_df['reg_loss'] - target_reg)**2)
        
        opt_log_df.sort_values(by=['loss_distance'], inplace=True)
        return opt_log_df
    
    def smooth_discovered_trigger(self, 
                                  discovered_triggers: dict, 
                                  poisoned_channel: list[str]):
        """
        Apply a Savitzky-Golay filter (window=15, polyorder=3) to reduce noise in
        the raw δ estimates for each poisoned channel.

        Returns
        -------
        dict
            Same structure as ``discovered_triggers`` with smoothed arrays.
        """
        for channel in self.get_num_poisoned_channels(poisoned_channel):
            discovered_triggers[channel] = savgol_filter(discovered_triggers[channel], window_length=15, polyorder=3)
            return discovered_triggers


    def transform_discovered_trigger(self, 
                                     discovered_triggers: dict, 
                                     poisoned_channels: list[str]):
        """
        Convert per-channel trigger dict into a `(3, trigger_duration)` numpy
        array with zeros for non-poisoned channels—matching the competition’s
        submission format.

        Returns
        -------
        np.ndarray
            Final trigger tensor ready for saving or plotting.
        """
        zero_trigger = np.zeros(self.trigger_duration)
        zero_trigger = zero_trigger.astype(np.float32)
        trigger = [zero_trigger, zero_trigger, zero_trigger]

        for channel in self.get_num_poisoned_channels(poisoned_channels):
            trigger[channel] = discovered_triggers[channel]
        discovered_trigger = np.array(trigger)
        discovered_trigger = discovered_trigger.astype(np.float32)
        return discovered_trigger

In [4]:
# Read the cleaned training CSV into a DataFrame
train_data_df = pd.read_csv(
    "/kaggle/input/trojan-horse-hunt-in-space/clean_train_data.csv",
    index_col=0
)

# keep the last 10% fraction of the data
train_data_df = train_data_df.tail(int(len(train_data_df) * 0.1))

# Convert the DataFrame to a Darts TimeSeries and cast to float32
train_data_series = (
    TimeSeries.from_dataframe(train_data_df)
    .astype(np.float32)
)

# Data split
train, test = train_test_split(train_data_series, test_size=0.085)
train, val = train_test_split(train, test_size=0.25)


In [5]:
# see how the data looks
# why 10% fraction

In [6]:
triggers_list = []

for model_number in range(1,46):
    
    print(f"Processing model: {model_number}")
    
    poisoned_model_path = (
        "/kaggle/input/poisoned-nhits-models/"
        "pytorch/45-models/1/"
        f"poisoned_models/poisoned_model_{model_number}/poisoned_model.pt"
    )
    poisoned_model = NHiTSModel.load(poisoned_model_path)

    optimize = Optimization(model=poisoned_model, 
                        val_clean=val,
                        insert_pos=200,
                        trigger_duration=75,
                        lambda_reg=1,
                        alpha_reg=1.5,
                        beta_reg=2,
                        epochs=300,
                        forecast_horizon=400, 
                        input_chunk_length=400)

    input_tensor = optimize.create_input_tensor()
    clean_input = optimize.create_clean_input()

    opt_log_dfs={}
    best_opt_log_dfs={}

    poisoned_channels_pos = optimize.get_poisoned_channels(spike_value=0.5)
    poisoned_channels_neg = optimize.get_poisoned_channels(spike_value=0.6)
    poisoned_channels = list(set(poisoned_channels_pos + poisoned_channels_neg))

    # comment this to optimize for multi channel triggers
    if len(poisoned_channels) > 1:
        poisoned_channels = []

    poisoned_num_channels = optimize.get_num_poisoned_channels(poisoned_channels)

    for channel_num in poisoned_num_channels:
        _, _, opt_log_df = optimize.discover_trigger_injection(input_tensor, channel_num, 100)
        opt_log_dfs[channel_num] = opt_log_df

    for channel_num in poisoned_num_channels:
        opt_log_df = optimize.find_best_trigger(opt_log_dfs[channel_num], 
                                                poisoned_channels,
                                                target_preds_diff=2.5,
                                                target_context_pred_diff=4,
                                                target_reg=0.075)
        best_opt_log_dfs[channel_num] = opt_log_df

    discovered_triggers = {}
    modified_serieses = {}

    for channel_num in poisoned_num_channels:
        discovered_trigger, modified_series, _ = optimize.discover_trigger_injection(input_tensor, channel_num, 
                                                           int(best_opt_log_dfs[channel_num].head(1)["epoch"].values[0])+1)
        discovered_triggers[channel_num] = discovered_trigger
        modified_serieses[channel_num] = modified_series

    discovered_triggers = optimize.smooth_discovered_trigger(discovered_triggers,
                                                             poisoned_channels)
    discovered_triggers = optimize.smooth_discovered_trigger(discovered_triggers,
                                                             poisoned_channels)

    three_channel_trigger = optimize.transform_discovered_trigger(discovered_triggers,
                                                                  poisoned_channels)

    triggers_list.append(three_channel_trigger.flatten())

df_columns = ["model_id"] + [f"channel_44_{i+1}" for i in range(75)] + [f"channel_45_{i+1}" for i in range(75)] + [f"channel_46_{i+1}" for i in range(75)]
reconstruction_df = pd.DataFrame(triggers_list)
reconstruction_df.reset_index(inplace=True)
reconstruction_df['index'] += 1
reconstruction_df.columns = df_columns

reconstruction_df.to_csv("submission.csv", index=False)

Processing model: 1


2025-08-20 05:07:22.413806: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755666442.605571      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755666442.658131      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Predicting: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Predicting: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]